In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import disk  # 用于绘制圆形的函数
from skimage.transform import radon, iradon  # Radon变换和反投影算法
from mpl_toolkits.mplot3d import Axes3D  # 3D绘图工具
from skimage.measure import marching_cubes  # Marching Cubes 用于提取3D等值面
from scipy.ndimage import zoom  # 用于缩放图像

# 设置扫描参数
image_size = 128  # 图像大小（像素）
num_angles = 180  # 投影角度数
num_slices = 64  # Z轴层数（体积深度）
slice_thickness = 1.0  # 采集层厚（毫米）
slice_gap = 0.2  # 层间距（毫米）
pitch = 1.0  # 螺距（影响纵向分辨率）
reconstruction_matrix = (128, 128)  # 重建图像的尺寸（矩阵大小）
reconstruction_slice_thickness = 1.0  # 重建层厚（毫米）

# 生成一个简单的几何体（圆柱体）
def generate_cylinder(image_size, num_slices):
    """生成一个简单的圆柱体3D模型"""
    cylinder = np.zeros((image_size, image_size, num_slices), dtype=np.float32)  # 创建一个全0的三维数组，表示体积图像
    center = (image_size // 2, image_size // 2)  # 圆柱体的中心位置
    radius = 30  # 圆柱体的半径
    for z in range(num_slices):  # 遍历每个切片
        rr, cc = disk(center, radius, shape=(image_size, image_size))  # 计算圆形的像素坐标
        cylinder[rr, cc, z] = 1.0  # 在当前切片上绘制圆形，值设为1，表示体积中的物质
    return cylinder  # 返回生成的圆柱体模型

# 生成螺旋CT扫描数据（Sinogram）
def generate_helical_scan(cylinder, pitch, slice_thickness):
    """模拟螺旋CT扫描过程"""
    num_slices = cylinder.shape[2]  # 获取体积深度（切片数）
    num_angles = 180  # 设置投影的角度数
    sinogram = np.zeros((num_slices, num_angles), dtype=np.float32)  # 创建一个空的Sinogram数组
    
    # 计算投影角度
    theta = np.linspace(0., 180., num_angles, endpoint=False)  # 在0到180度之间均匀生成投影角度

    # 模拟沿Z轴的螺旋扫描
    for z in range(num_slices):  # 遍历每个切片
        image_slice = cylinder[:, :, z]  # 获取当前切片图像
        sinogram[z, :] = np.sum(image_slice, axis=0)  # 对每个切片沿列（水平）方向求和，得到投影数据
    return sinogram, theta  # 返回生成的Sinogram数据和投影角度

# FBP重建
def fbp_reconstruction(sinogram, theta, image_size):
    """使用FBP算法重建图像"""
    reconstructed_image = iradon(sinogram, theta=theta, filter_name='ramp', circle=True)  # 使用FBP反投影算法重建图像
    return reconstructed_image  # 返回重建的图像

# 3D体积重建和可视化
def volume_rendering(cylinder, reconstruction_matrix):
    """3D体积渲染"""
    fig = plt.figure(figsize=(10, 8))  # 创建一个新的图形
    ax = fig.add_subplot(111, projection='3d')  # 创建3D坐标系
    verts, faces, _, _ = marching_cubes(cylinder, level=0.5)  # 使用Marching Cubes提取体积中的等值面
    ax.plot_trisurf(verts[:, 0], verts[:, 1], faces, verts[:, 2], cmap='viridis', lw=1)  # 在3D坐标系中绘制表面
    plt.show()  # 显示3D图像

# 主程序
cylinder = generate_cylinder(image_size, num_slices)  # 生成圆柱体3D模型
sinogram, theta = generate_helical_scan(cylinder, pitch, slice_thickness)  # 生成CT扫描数据
reconstructed_image = fbp_reconstruction(sinogram, theta, image_size)  # 使用FBP重建图像

# 可视化2D重建图像
plt.figure(figsize=(8, 8))  # 创建一个新的图形
plt.title("Reconstructed Image (2D FBP)")  # 设置标题
plt.imshow(reconstructed_image, cmap='gray')  # 显示2D重建图像
plt.axis('off')  # 关闭坐标轴
plt.show()  # 显示图像

# 进行3D体积渲染
volume_rendering(cylinder, reconstruction_matrix)  # 调用体积渲染函数，展示3D图像
